In [1]:
##read Data Base
from pymongo import MongoClient
from bson.objectid import ObjectId
uri = "mongodb://localhost:27017/database" #mongodb://<user_name>:<user_password>@ds<xxxxxx>.mlab.com:<xxxxx>/<database_name>
conn = MongoClient(uri)
db = conn.PaperData
#Spectrogram_Collection  =  db.get_collection('AudioSpectrogram_100thTag')
LineCNN_Collection  =  db.get_collection('LineCNNMatrix_100thTag_80p')

In [2]:
#db.collection_names()

In [3]:
#Lyric_Para
_num_Lines = 80
_num_LineLen = 25
_num_WEDim = 100
_num_LyricCNN_kernalSize = 3 # #CNN Size start from 3
_num_LyricCNN_kernalnum = 100 # for each size of kernal, how many different kernals 
##3 4 5 kernal size , 100 for each is from Convolutional Neural Network for Sententce Classification

In [4]:
#Tag para
_num_Tags = 50
_num_neural_TagFC1 = 512
_num_neural_TagFC2 = 256
_num_neural_TagFC3 = 2 #for soft max

In [5]:
_num_Tags = 50
_epchos = 200
_stopEpcho = 20
_batchSize = 15

In [6]:
import json 
import codecs
import numpy as np
def read_json(filename):
    with codecs.open(filename,'r',encoding = 'utf8') as infile:
        return np.array(json.load(infile))
    
x_train = read_json('x_train_'+str(_num_Tags)+'d_80p_sub.json')
x_test = read_json('x_test_'+str(_num_Tags)+'d_80p_sub.json')
y_train = read_json('y_train_'+str(_num_Tags)+'d_80p_sub.json')
y_test = read_json('y_test_'+str(_num_Tags)+'d_80p_sub.json')

In [7]:
len(x_train)

20540

In [8]:
from keras.layers import *
from keras.models import Model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [9]:
### Submodel for CRNN 

##Here is the sub model for timeDistribute layer in main model

# the share layer for all time distribution CNN
Lines_Input = Input(name= 'TextInput',batch_shape=(None,_num_LineLen, _num_WEDim, 1)) #(None,25,100,1)
LineFeature_List = []
for i in range(_num_LyricCNN_kernalSize):
    #result_shape = (timestep, sentencelen(num of CNN result for each kernal), WE_dim, num_kernal) = (None,25,100,32)
    CL  = Conv2D(_num_LyricCNN_kernalnum, kernel_size = (i+3,_num_WEDim), padding='same')(Lines_Input) 
    AL = Activation('relu')(CL)
    BN = BatchNormalization()(AL)
    #max pooling for every kernal, result_shape = (None, 1, 1, 32)
    MPL = MaxPooling2D(_num_LineLen,_num_WEDim)(BN)
    MP_shape =  tf.shape(MPL)
    #flatten for matching GRU input size
    FL  = Flatten()(MPL) #result = (timestep size , )
    LineFeature_List.append(FL)
    
LineFeature = concatenate(LineFeature_List)
TextCNNMode = Model(inputs = Lines_Input, outputs = LineFeature)

In [10]:
TextCNNMode.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
TextInput (InputLayer)          (None, 25, 100, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 25, 100, 100) 30100       TextInput[0][0]                  
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 25, 100, 100) 40100       TextInput[0][0]                  
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 25, 100, 100) 50100       TextInput[0][0]                  
__________________________________________________________________________________________________
activation

In [11]:
###Main model 
#Lyric feature extraction, CRNN part
Text_Input = Input(name='LyricInput', shape=(_num_Lines,_num_LineLen, _num_WEDim, 1)) #(batch_size, timestep, 25, 100 ,1)
TD_CNN = TimeDistributed(TextCNNMode)(Text_Input)
GRU_F = GRU(_num_LyricCNN_kernalSize*_num_LyricCNN_kernalnum, return_sequences=False, kernel_initializer='he_normal', name='GRU_forward')(TD_CNN)
GRU_B = GRU(_num_LyricCNN_kernalSize*_num_LyricCNN_kernalnum, return_sequences=False, kernel_initializer='he_normal', name='GRU_backward', go_backwards=True)(TD_CNN)
LF = concatenate([GRU_F, GRU_B])

#pseudo-model for sheck summary
Softmax = Dense(_num_Tags, kernel_initializer='he_normal',activation='sigmoid')(LF)
Main_model = Model(inputs=Text_Input, outputs=Softmax)

In [12]:
Main_model.summary(120)

________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
LyricInput (InputLayer)                (None, 80, 25, 100, 1)     0                                                     
________________________________________________________________________________________________________________________
time_distributed_1 (TimeDistributed)   (None, 80, 300)            121500        LyricInput[0][0]                        
________________________________________________________________________________________________________________________
GRU_forward (GRU)                      (None, 300)                540900        time_distributed_1[0][0]                
________________________________________________________________________________________________________________________
GRU_backward (GRU)              

In [13]:
from keras.utils import Sequence
import json
import codecs
class LyricDataGenerator(Sequence):
    #batch size can only be 1
    def __init__(self, text_filenames, labels,batch_size):
        self.text_filenames, self.labels = text_filenames, labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.text_filenames)/ (self.batch_size)))

    def __getitem__(self, idx):
        #Here, you have to imprement what the data looks like in each epcho
        filename_List =  self.text_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        #print(filename_List)
        Text_batch_x = [] 
        for filename in filename_List:  
            Text_x = np.array(LineCNN_Collection.find_one({"Filename":filename})['LineMatrix'])
            Text_x = Text_x.reshape(_num_Lines,_num_LineLen, _num_WEDim,1) #(num_sentence, 20 word ,100dim WE, 1for cnn)
            Text_batch_x.append(Text_x)
            
        y_List = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        return np.array(Text_batch_x),np.array(y_List)
    def getitem(self, idx):
        #Here, you have to imprement what the data looks like in each epcho
        
        return self.__getitem__(idx)

In [14]:
LDG_train = LyricDataGenerator(x_train, y_train,_batchSize)
LDG_test = LyricDataGenerator(x_test, y_test,_batchSize)

In [15]:
x,y = LDG_train.getitem(0)

In [16]:
x.shape

(15, 80, 25, 100, 1)

In [17]:
y.shape

(15, 50)

In [18]:
##This AUC callback is for STL

from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class AUC_callback(Callback):
    def __init__(self,y,SDG,log_name):
        self.y = y
        self.SDG = SDG
        self.log_name = log_name
        
    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        Skip_Tag = []
        
        y_pred = self.model.predict_generator(self.SDG) # (#data x 50()) 
        #print(y_pred.shape,self.y.shape,len(self.SDG.text_filenames))
        for i in range(len(y_pred)):
            for j in range(len(y_pred[i])):
                if y_pred[i][j] >= 0.5 : 
                    y_pred[i][j] = 1
                else:
                    y_pred[i][j] = 0
        
        #print(y_pred.shape,self.y.shape)
        
        
        AUC_this_epoch = roc_auc_score(self.y ,y_pred)
        logs[self.log_name] = AUC_this_epoch
        print('%s : %s '% (self.log_name,(str(round(AUC_this_epoch,5)))))
        
    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [19]:
AUC_test_callback =AUC_callback(y_test,LDG_test,'AUC_test')
AUC_train_callback =AUC_callback(y_train,LDG_train,'AUC_train')

In [20]:
import time
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.epoch_time_end = time.time()
        self.times.append(self.epoch_time_end - self.epoch_time_start)
        logs['Timer'] = self.epoch_time_end - self.epoch_time_start
time_callback = TimeHistory()

In [21]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=_stopEpcho, verbose=1)

In [22]:
def multitask_loss(y_true, y_pred):
    # Avoid divide by 0
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Multi-task loss
    return K.mean(K.sum(- y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred), axis=1))

In [23]:
num_training_samples = len(x_train)
num_validation_samples = len(x_test)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from keras.utils import multi_gpu_model
parallel_model = multi_gpu_model(Main_model, gpus=2)
parallel_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
History = parallel_model.fit_generator(
                    generator= LDG_train,
                    steps_per_epoch=(num_training_samples // _batchSize),
                    epochs= _epchos,
                    verbose=1,
                    validation_data= LDG_test,
                    validation_steps=(num_validation_samples // _batchSize),
                    workers=12, use_multiprocessing=True,
                    callbacks = [early_stopping,time_callback]
                    )

Epoch 1/200
1368/1369 [============================>.] - ETA: 0s - loss: 0.4069 - acc: 0.8473

In [70]:
smallLDG_train = LyricDataGenerator(x_train[:10], y_train[:10],_batchSize)

In [71]:
result = parallel_model.predict_generator(smallLDG_train)

In [74]:
result[1]

array([0.64109516, 0.28912848, 0.3879006 , 0.31265792, 0.08526577,
       0.17852658, 0.27549997, 0.24905545, 0.08435257, 0.20332344,
       0.21299836, 0.04055744, 0.22828299, 0.32872945, 0.11536492,
       0.12245547, 0.12674901, 0.10242686, 0.21375282, 0.05019374,
       0.17104422, 0.29658225, 0.05775585, 0.18073812, 0.11680882,
       0.16215992, 0.14801858, 0.13594046, 0.2005305 , 0.00577888,
       0.09297061, 0.12293613, 0.13715751, 0.06519513, 0.16766681,
       0.08393165, 0.15374486, 0.03631251, 0.12070651, 0.09195225,
       0.15188475, 0.09428295, 0.1248062 , 0.00782196, 0.05357476,
       0.04428873, 0.10017511, 0.07161396, 0.04204907, 0.04719784],
      dtype=float32)

In [78]:
result[8]

array([0.63993925, 0.2862898 , 0.38756743, 0.31537664, 0.08484082,
       0.17749329, 0.27299446, 0.24521416, 0.08285759, 0.2019541 ,
       0.21280228, 0.04040561, 0.22881104, 0.32752085, 0.1154738 ,
       0.12185775, 0.12630685, 0.10211127, 0.21303959, 0.04968839,
       0.172335  , 0.2969863 , 0.05777103, 0.1811137 , 0.11492516,
       0.16255146, 0.14633226, 0.13628688, 0.19908823, 0.00579213,
       0.09253025, 0.12371977, 0.13751687, 0.06520582, 0.1664405 ,
       0.08399924, 0.1513631 , 0.03639603, 0.12148111, 0.09216613,
       0.15021965, 0.09303892, 0.12490935, 0.00764366, 0.05380074,
       0.0440358 , 0.10068581, 0.07093697, 0.04154344, 0.04676229],
      dtype=float32)

In [24]:
result = parallel_model.predict_generator(smallLDG_train)

In [25]:
result[0]

array([0.4085789 , 0.20232825, 0.22121426, 0.2251865 , 0.08814202,
       0.1351604 , 0.15566503, 0.12493239, 0.04917085, 0.12122378,
       0.10211617, 0.03356805, 0.14342067, 0.12755032, 0.18857175,
       0.06778888, 0.1100345 , 0.11518778, 0.10447442, 0.05100651,
       0.13747   , 0.09994594, 0.03858612, 0.12264695, 0.08557823,
       0.0793603 , 0.07214601, 0.08023074, 0.09906282, 0.01274512,
       0.1036813 , 0.07072857, 0.1084598 , 0.07173244, 0.07459857,
       0.05618154, 0.07240815, 0.03245007, 0.07520631, 0.06049702,
       0.07141275, 0.05335618, 0.06614666, 0.01751685, 0.06898241,
       0.03130869, 0.0832616 , 0.04428142, 0.03803425, 0.04306855],
      dtype=float32)

In [28]:
result[6]

array([0.44173026, 0.2374732 , 0.23722392, 0.22026414, 0.08128575,
       0.15096708, 0.18695475, 0.14923702, 0.060594  , 0.12776606,
       0.11899145, 0.03281648, 0.14720565, 0.14632595, 0.18456192,
       0.06724853, 0.12377871, 0.13132325, 0.11808017, 0.06334044,
       0.1342258 , 0.10744365, 0.03601804, 0.12794243, 0.10678806,
       0.08363817, 0.08407989, 0.08699004, 0.11468057, 0.01077297,
       0.10378685, 0.07477808, 0.11557111, 0.06754161, 0.08978192,
       0.06166533, 0.08988193, 0.02571851, 0.08343556, 0.05272269,
       0.08044222, 0.06407141, 0.07051423, 0.0188871 , 0.07783408,
       0.03585695, 0.08384464, 0.05452531, 0.04334679, 0.05519047],
      dtype=float32)

In [29]:
from keras.models import model_from_json
json_string = parallel_model.to_json()
with codecs.open('Lyric_STL.json','w', encoding = 'utf8') as outfile:
    json.dump(json_string,outfile)
#parallel_model = model_from_json(json_string)

In [66]:
Main_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [67]:
History = Main_model.fit(x,y)

Epoch 1/1
10/10 [==============================] - 3s 313ms/step - loss: 0.7100 - acc: 0.5280


In [ ]:
History = Main_model.fit_generator(
                    generator= LDG_train,
                    #steps_per_epoch=(num_training_samples // _batchsize),
                    epochs= _epchos,
                    verbose=1,
                    validation_data= LDG_test,
                    #validation_steps=(num_validation_samples // _batchsize),
                    #workers=3, use_multiprocessing=True,
                    callbacks = [AUC_train_callback, AUC_test_callback,early_stopping,time_callback]
                    )

Epoch 1/100
 6/10 [=================>............] - ETA: 1:01 - loss: 0.5984 - acc: 0.8533

In [71]:
for filename in x_test:
    text = np.array(LineCNN_Collection.find_one({"Filename":filename})['LineMatrix'])
    if text.shape != (80,25,100) : 
        print(filename,text.shape)